Probabilistic Primality Tests
===============
Cyrus B.

April 2019

In [2]:
import numpy as np
from random import randint
import matplotlib.pyplot as plt
%matplotlib inline

### Modular Congruency

$a$ and $b$ are positive real integers


$ {a \equiv b \pmod{n}} $, reads "a is congruent to b".

This means that $a - b$ is divisible by $n$

### Fermat's Little Theorem

If $p$ is a prime number and $a$ is a positive integer less than $p$, _and_ $p$ does not divide $a$.


Then ${a^{p-1} \equiv 1 \pmod{p}}$

#### Quick Example

$ p = 7 $

$ a = 4 $

$ a^{p-1} = 4^{6} = 4096 $

$\frac{4096 - 1}{7} = 585$

### Fermat Primality Test

Fermat's little theorem can easily be used to find prime numbers. If we want to test the primality of a number, we simply plug it into the equation for $p$ then pick an $a$ such that $0 < a < p$ and p does not divide a. If p divides $a^{p-1}$ then we know p is (very likely) prime. $p$ is not necesarilly prime because there are some numbers, Carmichael numbers, that the Fermat Primality Test will identify as prime even though they are not. 

In [66]:
def fpt(p):
    """
    Takes a number p and determines if it is (probably) prime
    Linear time efficiency O(p)
    """
    if p == 2:
        return True
    
    a = 2
    exp = a
    """
    I am defining exponentiation in this way so that I can show
    the difference in speed between normal exponentiation and
    exponentiation by doubling
    """
    for _ in range(1, p - 1):
        exp *= a
        
    if exp % p == 1:
        return True
    return False

In [68]:
for i in range(2, 50):
    if fpt(i): print(i)

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47


In [ ]:
def fpt_fastexp(n):
    """
    A very nice way to speed up the FPT algorithm is to
    use modular exponentiation by doubling.
    """
    if p == 2:
        return True
    a = 2
    exp = a
    result = 1
    while n > 0:
        if n & 1:
            result = (result * exp) % n
        exp //= 2

Sources
=======

https://math.mit.edu/research/highschool/primes/materials/2014/Narayanan.pdf


https://en.wikipedia.org/wiki/Primality_test#Fermat_primality_test

https://en.wikipedia.org/wiki/Exponentiation_by_squaring#Basic_method

https://mathlesstraveled.com/2018/08/03/the-fermat-primality-test/